# Compound Events
- https://github.com/e-baumer/standard_precip
- https://github.com/e-baumer/standard_precip/blob/master/standard_precip/base_sp.py
- Handle missing data to calculate spi (last paragraph) https://www.droughtmanagement.info/literature/WMO_standardized_precipitation_index_user_guide_en_2012.pdf#page=9


### SPI:
- Add options
    - freq = ['D', 'M']
    - scale = #
- If freq == 'M': get monthly pr files
- If freq == 'D': get daily pr files
- Concat 3 dfs of pr data:
    - [historical, ssp126]
    - [historical, ssp245]
    - [historical, ssp370]
- Calculate SPI for each model using "from standard_precip.spi import SPI"
- Extract results JJA for 2015-2100 for spi and tasmax
    - Filter by common model/column names
    - Concat spi and tasmax axis=0
    - Concat ssp's axis=1
    - Determine if compound
    - Output 1 df per threshold combo

### Problem:
- KACE-1-0-G currently skipped since division by zero error (~1000 missing)
- SPI calc does not handle nan's well: 1 row of missing pr results in at least 30 rows of missing spi

- Why are there missing values?
    - KACE-1-0-G: no value for all 31st day of month
    - Models missing 37 values: no value for feb 29 (leap years)
        - [INM-CM4-8, INM-CM5-0, NorESM2-MM, NorESM2-LM, GFDL-ESM4, GISS-E2-1-G, FGOALS-g3, BCC-CSM2-MR, CMCC-ESM2, CESM2]
     
- How to handle missing values? 
    - Check original code
    - Options: interpolation by time, multivariate imputation (slow), back/front fill, ...
#### Missing value solution: interpolation by linear (df sorted by time)

In [1]:
%%time

from process import *
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from standard_precip.spi import SPI

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (15, 4)
plt.rcParams['figure.dpi'] = 300 # 600
plt.rcParams['font.size'] = 10
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['axes.linewidth'] = 0.1
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['grid.linewidth'] = 0.1

event = ['CWHE','CDHE'][1]
months = [6, 7, 8]
center='LARC'

freq, scale = 'D', 30

initialize(center, event, months, freq, scale)
results, compound, pr_spi, tm, pr_, tm_ = main()

Processing ssp126 spi (1981-01-01 00:00:00 - 2100-12-31 00:00:00)...
Processing ssp245 spi (1981-01-01 00:00:00 - 2100-12-31 00:00:00)...
Processing ssp370 spi (1981-01-01 00:00:00 - 2100-12-31 00:00:00)...
CPU times: user 38.8 s, sys: 1.36 s, total: 40.1 s
Wall time: 40.8 s


In [2]:
pr_spi

INM-CM4-8_pr  INM-CM5-0_pr  NorESM2-MM_pr  NorESM2-LM_pr  \
ssp    date                                                                   
ssp126 2021-06-01      0.000000      0.081689       0.000000       0.000000   
       2021-06-02      0.297987      8.573282       0.000000       0.000000   
       2021-06-03      0.462374      8.971298       0.000000       0.000000   
       2021-06-04      0.120682      0.630614       0.000000       0.094576   
       2021-06-05      0.639671      0.000000       3.440921       4.472532   
...                         ...           ...            ...            ...   
ssp370 2100-08-27     13.431886      6.918253       0.064743       0.091623   
       2100-08-28     16.964950      0.076420       0.131618       0.115677   
       2100-08-29     13.477572      0.029418       8.323636       2.779805   
       2100-08-30      7.680827      2.135523      16.355370      17.000638   
       2100-08-31      0.824887      6.535375      17.977280       2.859532   

                   MIROC6_pr  GFDL-ESM4_pr  MIROC-ES2L_pr  GISS-E2-1-G_pr  \
ssp    date                                                                 
ssp126 2021-06-01   0.000814      2.004056       2.367464        0.008973   
       2021-06-02   0.000000      0.000000       0.508515        0.347845   
       2021-06-03   0.000000      3.174705       0.072587        9.452589   
       2021-06-04  13.903137      0.507504       0.000169        2.450261   
       2021-06-05   2.895703      0.934759       0.019373        2.092968   
...                      ...           ...            ...             ...   
ssp370 2100-08-27   0.000000      1.345558      15.426550        2.007805   
       2100-08-28   0.000920      0.000000       0.431562        0.572648   
       2100-08-29   9.479126      0.000000       0.008801        5.638641   
       2100-08-30   2.276963      1.311583       0.000000        0.009842   
       2100-08-31   0.000000      0.222884       1.663876        0.001381   

                   FGOALS-g3_pr  MPI-ESM1-2-HR_pr  ...  MRI-ESM2-0_spi  \
ssp    date                                        ...                   
ssp126 2021-06-01      1.138105          2.315744  ...        1.944710   
       2021-06-02      0.963457          8.882218  ...        1.513595   
       2021-06-03      0.681418          2.714005  ...        1.009377   
       2021-06-04      2.285066          3.401442  ...        0.807251   
       2021-06-05      0.986643          5.269065  ...        0.840292   
...                         ...               ...  ...             ...   
ssp370 2100-08-27     14.151917          6.271328  ...        0.412934   
       2100-08-28      1.349138          0.000000  ...        0.441634   
       2100-08-29      0.000000          0.000000  ...        0.523189   
       2100-08-30      0.281959          0.000000  ...        0.607270   
       2100-08-31      3.322464          0.000000  ...        0.614858   

                   CMCC-ESM2_spi  ACCESS-ESM1-5_spi  EC-Earth3_spi  \
ssp    date                                                          
ssp126 2021-06-01      -0.067509           0.615972      -1.286553   
       2021-06-02      -0.528473           0.692402      -1.232343   
       2021-06-03      -0.554914           0.716502      -0.997270   
       2021-06-04      -0.569984           0.756254      -0.828589   
       2021-06-05      -0.642846           0.978917      -0.822810   
...                          ...                ...            ...   
ssp370 2100-08-27      -0.092322          -0.944546       1.536825   
       2100-08-28      -0.060102          -0.774077       1.321439   
       2100-08-29      -0.040056           0.146903       0.786670   
       2100-08-30      -0.022949           0.178776       0.679696   
       2100-08-31       0.315906           0.116805       0.547681   

                   ACCESS-CM2_spi  IPSL-CM6A-LR_spi  CNRM-ESM2-1_spi  \
ssp    date                                

In [6]:
metrics = ['_day_total$', '_event__total$', '_event_max$'] #>>>
display(results['spi<-1_tasmax>32.2'].filter(regex='ACCESS-CM2'))#.sum())

ACCESS-CM2_compound_event_max  ACCESS-CM2_compound_event__total  \
ssp    date                                                                    
ssp126 2021                              6                                 6   
       2022                             10                                43   
       2023                              0                                 0   
       2024                              3                                 3   
       2025                              0                                 0   
...                                    ...                               ...   
ssp370 2096                              3                                 3   
       2097                              6                                 8   
       2098                             13                                15   
       2099                              2                                 2   
       2100                              4                                 4   

             ACCESS-CM2_compound_day_total  
ssp    date                                 
ssp126 2021                              6  
       2022                             46  
       2023                              0  
       2024                              3  
       2025                              0  
...                                    ...  
ssp370 2096                              3  
       2097                              8  
       2098                             16  
       2099                              2  
       2100                              4  

[240 rows x 3 columns]

In [4]:
# for column in spi_dfs[ssp_name].columns[1:]:
#     spi_dfs[ssp_name][column].plot.kde()
#     plt.title(f'Precipitation')
# plt.xlabel('Models')
# plt.ylabel('Density')
# plt.show()